In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import Counter
 
def find_vector_set(diff_image, new_size):
    i = 0
    j = 0
    vector_set = np.zeros((int(new_size[0] * new_size[1] / 25), 25))
    while i < vector_set.shape[0]:
        while j < new_size[0]:
            k = 0
            while k < new_size[1]:
                block   = diff_image[j:j+5, k:k+5]
                feature = block.ravel()
                vector_set[i, :] = feature
                k = k + 5
            j = j + 5
        i = i + 1
 
    mean_vec   = np.mean(vector_set, axis = 0)
    vector_set = vector_set - mean_vec
    return vector_set, mean_vec
 
def find_FVS(EVS, diff_image, mean_vec, new):
    i = 2
    feature_vector_set = []
 
    while i < new[0] - 2:
        j = 2
        while j < new[1] - 2:
            block = diff_image[i-2:i+3, j-2:j+3]
            feature = block.flatten()
            feature_vector_set.append(feature)
            j = j+1
        i = i+1
 
    FVS = np.dot(feature_vector_set, EVS)
    FVS = FVS - mean_vec
    print("feature vector space size", FVS.shape)
    return FVS
 
def clustering(FVS, components, new):
    kmeans = KMeans(components, verbose = 0)
    kmeans.fit(FVS)
    output = kmeans.predict(FVS)
    count  = Counter(output)
 
    least_index = min(count, key = count.get)
    change_map  = np.reshape(output,(new[0] - 4, new[1] - 4))
    return least_index, change_map


In [2]:
import glob, os
import rasterio

for path in glob.glob('/mnt/Nam/public/Linh/mongolia/*.tar'):
    path = path.replace('.tar', '')
    names = os.path.basename(path)+"_SR_B"
    if len(os.listdir(path)):
        output = "/mnt/Nam/public/Linh/aaa/"+os.path.basename(path)+".tif"
        print(output)
        if not os.path.exists(output):
            file_list = []
            for i in glob.glob(path+'/*.TIF'):
                if names in i:
                    file_list.append(i)
            print(file_list)
            with rasterio.open(file_list[0]) as src0:
                meta = src0.meta
            meta.update(count = len(file_list))
            with rasterio.open(output, 'w', **meta) as dst:
                for id, layer in enumerate(file_list, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
    break

/mnt/Nam/public/Linh/aaa/LC09_L2SP_137024_20220328_20220330_02_T1.tif
['/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B1.TIF', '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B2.TIF', '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B3.TIF', '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B4.TIF', '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B5.TIF', '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B6.TIF', '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B7.TIF']


In [13]:
file_list = ['/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B1.TIF',
            '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B2.TIF',
            # '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B3.TIF',
            # '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B4.TIF',
            # '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B5.TIF',
            # '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B6.TIF',
            # '/mnt/Nam/public/Linh/mongolia/LC09_L2SP_137024_20220328_20220330_02_T1/LC09_L2SP_137024_20220328_20220330_02_T1_SR_B7.TIF',
            ]

output = output = "/mnt/Nam/public/Linh/aaa/stack.tif"

with rasterio.open(file_list[0]) as src0:
    meta = src0.meta
meta.update(count = len(file_list))
with rasterio.open(output, 'w', **meta) as dst:
    for id, layer in enumerate(file_list, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

In [14]:
meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': 0.0,
 'width': 7901,
 'height': 7981,
 'count': 1,
 'crs': CRS.from_epsg(32647),
 'transform': Affine(30.0, 0.0, 402885.0,
        0.0, -30.0, 5846115.0)}

In [8]:
import glob, os
import rasterio
import shutil
from tqdm import tqdm

for path in tqdm(glob.glob('/mnt/Nam/public/Linh/capital_mongolia/*.tar')):
    path = path.replace('.tar', '')
    names = os.path.basename(path)+"_SR_B"
    if len(os.listdir(path)):
        output = "/mnt/Nam/QUYET/"+os.path.basename(path)+".tif"
        file_list = []
        for i in glob.glob(path+'/*.TIF'):
            if names in i:
                file_list.append(i)
        with rasterio.open(file_list[0]) as src0:
            meta = src0.meta
            meta.update(count = len(file_list))
        with rasterio.open(output, 'w', **meta) as dst:
            for id, layer in enumerate(file_list, start=1):
                with rasterio.open(layer) as src1:
                    dst.write_band(id, src1.read(1))
        # shutil.copyfile(output, output.replace("/mnt/Nam/QUYET/", "/mnt/Nam/public/Linh/aaa/"))
        # os.remove(output)

100%|██████████| 17/17 [04:44<00:00, 16.75s/it]


In [1]:
import glob, os
import rasterio
from multiprocessing.pool import Pool
from functools import partial
import multiprocessing

list_id = glob.glob('/mnt/Nam/public/Linh/mongolia/*.tar')

def stack_image(path):
    path = path.replace('.tar', '')
    names = os.path.basename(path)+"_SR_B"
    if len(os.listdir(path)):
        output = "/mnt/Nam/QUYET/data/"+os.path.basename(path)+".tif"
        file_list = []
        for i in glob.glob(path+'/*.TIF'):
            if names in i:
                file_list.append(i)
        try:
            with rasterio.open(file_list[0]) as src0:
                meta = src0.meta
                meta.update(count = len(file_list))
            with rasterio.open(output, 'w', **meta) as dst:
                for id, layer in enumerate(file_list, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
        except:
            print(output)

core = multiprocessing.cpu_count()//2
p_cnt = Pool(processes=core)    
p_cnt.map(partial(stack_image), list_id)
p_cnt.close()
p_cnt.join()

/mnt/Nam/QUYET/data/LC09_L2SP_130030_20220327_20220329_02_T1.tif


In [7]:
import os
import rasterio
from rasterio.merge import merge
import glob, os
import rasterio
from multiprocessing.pool import Pool
from functools import partial
import multiprocessing
import json
from tqdm import tqdm

file_json = json.load(open("/home/skymap/Downloads/dict_img_fid_aoi.json"))

def stack_image(path):
    path = path.replace('.tar', '')
    names2 = os.path.basename(path)+"_SR_B2"
    names3 = os.path.basename(path)+"_SR_B3"
    names4 = os.path.basename(path)+"_SR_B4"
    if len(os.listdir(path)):
        output = "/mnt/Nam/QUYET/data/13/" +os.path.basename(path)+".tif"
        file_list = []
        for i in glob.glob(path+'/*.TIF'):
            if (names2 in i) or (names3 in i) or (names4 in i):
                file_list.append(i)
        with rasterio.open(file_list[0]) as src0:
            meta = src0.meta
            meta.update(count = len(file_list))
        with rasterio.open(output, 'w', **meta) as dst:
            for id, layer in enumerate(file_list, start=1):
                with rasterio.open(layer) as src1:
                    dst.write_band(id, src1.read(1))
                    
                    

for idx in tqdm(file_json["13.0"]):
    stack_image("/mnt/Nam/public/Linh/mongolia/"+idx)


100%|██████████| 69/69 [19:02<00:00, 16.56s/it]


In [13]:
import os
import rasterio
from rasterio.merge import merge

def write_image(data, height, width, numband, crs, tr, out, nodata=None):
    with rasterio.open(out, 'w', driver='GTiff', height=height, width=width, count=numband,
                        dtype=data.dtype, crs=crs, transform=tr, nodata=nodata,) as dst:
        dst.write(data)

def mosaic(dir_path, list_img_name, out_path, base_img=None):
    src_files_to_mosaic = []
    for name_f in list_img_name:
        fp = os.path.join(dir_path, name_f)
        src = rasterio.open(fp)
        src_files_to_mosaic.append(src)
    if base_img:
        print("Mosaic gdal with base image")
        src_files_to_mosaic.append(rasterio.open(base_img))
    else:
        print("Mosaic gadal without base image")
    mosaic, out_trans = merge(src_files_to_mosaic)
    write_image(mosaic, mosaic.shape[1], mosaic.shape[2], mosaic.shape[0], src.crs, out_trans, out_path, nodata=0)

dir_path = '/mnt/Nam/public/Linh/mosaic_demo'
# list_img_name = os.listdir(dir_path)
list_img_name = ["mosaic_4.tif", "mosaic_3.tif", "mosaic_2.tif", "mosaic_1.tif"]
out_path = "/mnt/Nam/public/Linh/mosaic_all.tif"
mosaic(dir_path, list_img_name[15:], out_path)

Mosaic gadal without base image


IndexError: list index out of range

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
imagepath1 = '/mnt/data/pre-processing/corpus/AndasolChangePair/Andasol_09051987.jpg'
imagepath2 = '/mnt/data/pre-processing/corpus/AndasolChangePair/Andasol_09122013.jpg'
# Read image
image1 = np.array(Image.open(imagepath1))
image2 = np.array(Image.open(imagepath2))

In [3]:
new_size = np.asarray(image1.shape) / 5
new_size = new_size.astype(int) * 5

diff_image = abs(image1 - image2)

vector_set, mean_vec = find_vector_set(diff_image, new_size)

In [4]:
pca     = PCA()
pca.fit(vector_set)
EVS = pca.components_

FVS     = find_FVS(EVS, diff_image, mean_vec, new_size)
components = 3
least_index, change_map = clustering(FVS, components, new_size)

change_map[change_map == least_index] = 255
change_map[change_map != 255] = 0

change_map = change_map.astype(np.uint8)
kernel     = np.asarray(((0,0,1,0,0),
                        (0,1,1,1,0),
                        (1,1,1,1,1),
                        (0,1,1,1,0),
                        (0,0,1,0,0)), dtype=np.uint8)
cleanChangeMap = cv2.erode(change_map,kernel)

feature vector space size (1430416, 25)


In [12]:
import glob, os
import numpy as np
import math
import sys
from pyproj import Proj, transform

def create_list_id(path):
    list_id = []
    os.chdir(path)
    for file in glob.glob("*.tif"):
        list_id.append(file[:-4])
        # print(file[:-4])
    return list_id

def main_split(foder_image,split,out_dir_data):
    import shutil

    image_list = create_list_id(foder_image)
    np.random.shuffle(image_list)
    count = len(image_list)
    cut_idx = int(round(count*split))
    print(cut_idx)
    train_list = image_list[0:cut_idx]
     
    # val_list = image_list[cut_idx:count]
    other_list = [id_image for id_image in image_list if id_image not in train_list]
    val_list = other_list
    path_train = os.path.join(out_dir_data,'train','images')
    if not os.path.exists(path_train):
        os.makedirs(path_train)
    path_val = os.path.join(out_dir_data,'val','images')
    if not os.path.exists(path_val):
        os.makedirs(path_val)
    path_test = os.path.join(out_dir_data,'test','images')
    # if not os.path.exists(path_test):
    #     os.makedirs(path_test)
    for image_name in train_list:
        # shutil.copy(os.path.join(foder_image,image_name+'.tif'), path_train)
        os.rename(os.path.join(foder_image,image_name+'.tif'), os.path.join(path_train,image_name+'.tif'))
    for image_name in val_list:
        # shutil.copy(os.path.join(foder_image,image_name+'.tif'), path_val)
        os.rename(os.path.join(foder_image,image_name+'.tif'), os.path.join(path_val,image_name+'.tif'))
    return path_train,path_val,path_test
    # return path_train,path_val
if __name__=='__main__':
    foder_image = "/mnt/Nam/Buildinggg/data_train_v1/image/"
# foder_image_mask = os.path.abspath(sys.argv[2])

    split = 0.8
    out_dir_data = "/mnt/Nam/Buildinggg/data_train_v1/"
    main_split(foder_image,split,out_dir_data)

0


In [20]:
import tqdm
import glob, os
import json, rasterio
import sys
import geopandas as gp
import numpy as np
from multiprocessing.pool import Pool
from functools import partial
import multiprocessing
import datetime
import cv2, math
from tqdm import *

core = multiprocessing.cpu_count()

def create_list_id(path):
    list_id = []
    os.chdir(path)
    for file in glob.glob("*.tif"):
        list_id.append(file[:-4])
        # print(file[:-4])
    return list_id

def convert_geography_to_pixel(x):
    _segmentation = []
    for a in x.exterior.coords:
        x = float(int((a[0] - transform[2])/ transform[0]))
        y = float(int((a[1] - transform[5])/ transform[4]))
        _segmentation.extend((x,y))
    return _segmentation

def areaes(x):
    return round((x.area/transform[0]/-transform[4]),2)

def boxes(x):
    _bbox = [float(int((x.bounds[3] - transform[5])/ transform[4])),
            float(int((x.bounds[0] - transform[2])/ transform[0])),
            float(math.ceil((x.bounds[1] - x.bounds[3])/ transform[4])),
            float(math.ceil((x.bounds[2] - x.bounds[0])/ transform[0]))]
    return _bbox
    
def create_list_annotation(image_name,shape_dir,image_dir):
    with rasterio.open(os.path.join(image_dir, image_name+".tif")) as r:
        global transform
        transform = r.transform
        height, width = r.height, r.width
        projstr = r.crs.to_string()
    shapefile = gp.read_file(os.path.join(shape_dir, image_name+'.shp'))
    shapefile = shapefile.to_crs(projstr)

    list_cnt = shapefile["geometry"].apply(convert_geography_to_pixel)
    area = shapefile["geometry"].apply(areaes)
    box = shapefile["geometry"].apply(boxes)
    return height, width, list_cnt, area, box

def main_gen_anotation(image_dir,shape_dir):
    parent = os.path.dirname(image_dir)
    _final_object = {}
    _final_object["info"]= {
                            "contributor": "crowdAI.org",
                            "about": "Dataset for crowdAI Mapping Challenge",
                            "date_created": datetime.datetime.utcnow().isoformat(' '),
                            "description": "crowdAI mapping-challenge dataset",
                            "url": "https://www.crowdai.org/challenges/mapping-challenge",
                            "version": "1.0",
                            "year": 2018
                            }
                        
    _final_object["categories"]=[
                    {
                        "id": 100,
                        "name": "building",
                        "supercategory": "building"
                    }
                ]
    date_captured=datetime.datetime.utcnow().isoformat(' ')
    license_id=1
    coco_url=""
    flickr_url=""
    _images = []
    _annotations = []
    _list_image = create_list_id(image_dir)
    _image_id = 0
    _annotation_id = 0
    with tqdm(total=len(_list_image)) as pbar:
        for _image_name in _list_image:
            pbar.update()
            _image_id = _image_id + 1
            _file_name = _image_name + '.tif'
            _height, _width, list_segmentation, list_area, list_box = create_list_annotation(_image_name,shape_dir,image_dir)
            image_info = {
                        "id": _image_id,
                        "file_name": _file_name,
                        "width": _width,
                        "height": _height,
                        "date_captured": date_captured,
                        "license": license_id,
                        "coco_url": coco_url,
                        "flickr_url": flickr_url
                        }
            _images.append(image_info)
            for idxxx, _segmentation in enumerate(list_segmentation):
                _annotation_id = _annotation_id + 1
                annotation_info = {
                    "id": _annotation_id,
                    "image_id": _image_id,
                    "category_id": 100,
                    "iscrowd": 0,
                    "area": list_area[idxxx],
                    "bbox": list_box[idxxx],
                    "segmentation": _segmentation,
                    "width": _width,
                    "height": _height,
                }
                _annotations.append(annotation_info)
    _final_object["images"]=_images
    _final_object["annotations"]=_annotations
    fp = open(os.path.join(parent,'annotation.json'), "w")
    print("Writing JSON...")
    fp.write(json.dumps(_final_object))
    print("Done!")

if __name__ == "__main__":
    image_dir = "/mnt/Nam/Buildinggg/data_train_v1/train/images"
    shape_dir = "/mnt/Nam/Buildinggg/data_train_v1/shape/"
    main_gen_anotation(image_dir,shape_dir)


100%|██████████| 6770/6770 [03:44<00:00, 30.19it/s]


Writing JSON...
Done!


In [10]:
import tqdm
import glob, os
import json, rasterio
import sys
import geopandas as gp
import numpy as np
from multiprocessing.pool import Pool
from functools import partial
import multiprocessing
import datetime
import cv2, math
from tqdm import *
a = json.load(open("/mnt/Nam/Buildinggg/data_train_v1/val/annotation.json"))

In [17]:
xxx = pd.DataFrame(a['images'])
xxx

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,varsaivira3_5469.tif,576,576,2022-04-29 03:49:22.810911,1,,
1,2,53394622_3388.tif,640,640,2022-04-29 03:49:22.810911,1,,
2,3,Mt_Isa_2017_10cm_Mosaic_2952.tif,1536,1536,2022-04-29 03:49:22.810911,1,,
3,4,10300100B0A32C00-visual_8120.tif,512,512,2022-04-29 03:49:22.810911,1,,
4,5,madrid_5835.tif,512,512,2022-04-29 03:49:22.810911,1,,
...,...,...,...,...,...,...,...,...
1688,1689,10seg805325_1400.tif,512,512,2022-04-29 03:49:22.810911,1,,
1689,1690,varsaivira_4742.tif,576,576,2022-04-29 03:49:22.810911,1,,
1690,1691,varsaivira_4972.tif,576,576,2022-04-29 03:49:22.810911,1,,
1691,1692,10300100BB7A5700-visual_7951.tif,512,512,2022-04-29 03:49:22.810911,1,,


In [18]:
xxx[xxx['file_name']=="Visalia_15t1_2018_NAD83_3_4_4326_2523.tif"]

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
8,9,Visalia_15t1_2018_NAD83_3_4_4326_2523.tif,1024,1024,2022-04-29 03:49:22.810911,1,,


In [11]:
import pandas as pd
h = pd.DataFrame(a['annotations'])
h

,id,image_id,category_id,iscrowd,area,bbox,segmentation,width,height
0,1,1,100,0,342.03,"[569.0, 303.0, 7.0, 57.0]","[303.0, 570.0, 359.0, 569.0, 359.0, 575.0, 303...",576,576
1,2,1,100,0,783.13,"[550.0, 0.0, 26.0, 34.0]","[0.0, 550.0, 33.0, 553.0, 31.0, 575.0, 0.0, 57...",576,576
2,3,1,100,0,1594.53,"[534.0, 388.0, 42.0, 64.0]","[388.0, 562.0, 431.0, 534.0, 451.0, 568.0, 439...",576,576
3,4,1,100,0,955.04,"[540.0, 168.0, 36.0, 47.0]","[168.0, 562.0, 201.0, 540.0, 214.0, 562.0, 194...",576,576
4,5,1,100,0,1239.74,"[529.0, 231.0, 47.0, 44.0]","[231.0, 542.0, 259.0, 529.0, 275.0, 566.0, 255...",576,576
...,...,...,...,...,...,...,...,...,...
56755,56756,1693,100,0,2965.88,"[0.0, 0.0, 94.0, 45.0]","[7.0, 0.0, 15.0, 2.0, 16.0, 5.0, 38.0, 11.0, 4...",512,512
56756,56757,1693,100,0,1048.66,"[0.0, 476.0, 34.0, 36.0]","[512.0, 30.0, 483.0, 33.0, 482.0, 19.0, 476.0,...",512,512
56757,56758,1693,100,0,2220.33,"[0.0, 88.0, 37.0, 87.0]","[175.0, 0.0, 166.0, 36.0, 92.0, 17.0, 88.0, 9....",512,512
56758,56759,1693,100,0,39.80,"[0.0, 28.0, 5.0, 19.0]","[47.0, 0.0, 46.0, 4.0, 28.0, 0.0, 47.0, 0.0]",512,512


In [19]:
h[h["image_id"]==9]

,id,image_id,category_id,iscrowd,area,bbox,segmentation,width,height
388,389,9,100,0,1043.60,"[924.0, 0.0, 71.0, 30.0]","[0.0, 924.0, 29.0, 978.0, 0.0, 994.0, 0.0, 924.0]",1024,1024
389,390,9,100,0,3937.19,"[866.0, 458.0, 89.0, 45.0]","[458.0, 867.0, 503.0, 866.0, 503.0, 955.0, 458...",1024,1024
390,391,9,100,0,18697.58,"[800.0, 726.0, 194.0, 195.0]","[867.0, 913.0, 812.0, 993.0, 726.0, 941.0, 811...",1024,1024
391,392,9,100,0,6536.95,"[817.0, 143.0, 125.0, 57.0]","[196.0, 817.0, 199.0, 940.0, 146.0, 942.0, 143...",1024,1024
392,393,9,100,0,20996.29,"[660.0, 801.0, 193.0, 215.0]","[941.0, 661.0, 1016.0, 703.0, 930.0, 853.0, 80...",1024,1024
393,394,9,100,0,11709.93,"[680.0, 465.0, 130.0, 123.0]","[523.0, 683.0, 586.0, 680.0, 587.0, 744.0, 561...",1024,1024
394,395,9,100,0,1119.98,"[623.0, 616.0, 27.0, 46.0]","[616.0, 625.0, 660.0, 623.0, 661.0, 649.0, 618...",1024,1024
395,396,9,100,0,12087.28,"[567.0, 14.0, 84.0, 175.0]","[125.0, 626.0, 126.0, 648.0, 15.0, 650.0, 14.0...",1024,1024
396,397,9,100,0,5890.62,"[555.0, 944.0, 132.0, 80.0]","[1001.0, 687.0, 944.0, 613.0, 1023.0, 555.0, 1...",1024,1024
397,398,9,100,0,9115.69,"[547.0, 450.0, 90.0, 125.0]","[451.0, 606.0, 450.0, 547.0, 571.0, 548.0, 574...",1024,1024


In [1]:
import glob
import rasterio
import numpy as np
import glob, os
import sys
from multiprocessing.pool import Pool
from functools import partial
import multiprocessing
import argparse
from itertools import product
import rasterio
from rasterio import windows
import fiona
import pandas as pd
import pprint
from shapely.geometry import Polygon, box, mapping, Point, LineString
from shapely.strtree import STRtree
import datetime
import geopandas as gp
import copy
import math
import json
from tqdm import tqdm

def get_tiles(ds, width, height):
    ncols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, ncols, 600), range(0, nrows, 600))
    big_window = windows.Window(col_off=0, row_off=0, width=ncols, height=nrows)
    offset = []
    for col_off, row_off in offsets:
        if row_off + _width > nrows:
            row_off = nrows - _width
        if  col_off + _height > ncols:
            col_off = ncols - _height
        offset.append((col_off, row_off))
    offset = set(offset)
    for col_off, row_off in tqdm(offset): 
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform
        
image_path = '/media/skymap/Nam/tmp_Nam/GGHL/data/testt'
if not os.path.exists(image_path):
    os.makedirs(image_path)
output_box = 'box_{}'

img_path = '/media/skymap/Nam/tmp_Nam/GGHL/data/testt.tif'

idx = len(os.listdir(image_path))
with rasterio.open(img_path) as inds:
    _width, _height = 800, 800
    meta = inds.meta.copy()
    out_crs = inds.crs.to_string()

    for window, transform in get_tiles(inds, _width, _height):
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        outpath_image = os.path.join(image_path, output_box.format('{0:0004d}'.format(idx)))+ '.tif'
            
        with rasterio.open(outpath_image, "w", **meta, compress='lzw') as dest:
            dest.write(inds.read(window=window))
        idx+=1

100%|██████████| 30/30 [00:00<00:00, 30.81it/s]


In [32]:
import numpy as np
import geopandas, glob, os, rasterio, cv2

path_images = "/mnt/Nam/Buildinggg/data_train_v1/train/images/*.tif"
path_shapes = "/mnt/Nam/Buildinggg/data_train_v1/shape/"

def convert(a):
    return ((np.array(a.exterior.coords[:])[...,:2] - np.array((transform[2], transform[5])))/ np.array((transform[0], transform[4]))).astype(np.uint16)

def annToMask(ann, height, width):
    m = np.zeros((height,width), dtype=np.uint8)
    cv2.fillPoly(m, ann.astype(int), 1)
    return m


path_image = "/mnt/Nam/Buildinggg/data_train_v1/train/images/madrid_5968.tif"
path_shape = os.path.join(path_shapes, os.path.basename(path_image).replace(".tif",'.shp'))
with rasterio.open(path_image) as r:
    transform = r.transform
    width, height = r.width, r.height
    img = r.read().transpose(1,2,0)
img_1 = img.copy()
gdf = geopandas.read_file(path_shape)

data = gdf['geometry'].apply(convert)
for i in data:
    datax = annToMask(i[np.newaxis,...], width, height)
    cnts, hierarchy = cv2.findContours((255*datax).astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rect = cv2.minAreaRect(cnts[0])
    if rect[1][1]>rect[1][0]:
        angle = 90-rect[2]
    else:
        angle = -rect[2]
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img_1,[box],0,(255,0,0),2)

In [23]:
a = gdf['geometry'][0]

In [24]:
((np.array(a.exterior.coords[:]) - np.array((transform[2], transform[5])))/ np.array((transform[0], transform[4]))).astype(np.uint16)

ValueError: operands could not be broadcast together with shapes (4,3) (2,) 

In [29]:
np.array(a.exterior.coords[:])[...,:2]

array([[ 444293.04905176, 4475635.2       ],
       [ 444294.39338828, 4475637.53821257],
       [ 444298.63138209, 4475635.2       ],
       [ 444293.04905176, 4475635.2       ]])